In [1]:
from sympy.physics.quantum.qubit import Qubit
from sympy import Matrix, sqrt, symbols, simplify
import sympy as sp
from IPython.display import display, Latex
from itertools import product

In [2]:
from sympy.physics.quantum import Ket, Bra, Dagger, represent, TensorProduct

In [3]:
import random
import math
import cmath

In [4]:
import itertools

In [5]:
# Step 1: Symbols define karein
alpha, beta = symbols('α β')

# Step 2: Quantum state banayein (example: superposition state)
psi = alpha * Qubit('0') + beta * Qubit('1')

# Step 3: Density matrix calculate karein
rho = psi * Dagger(psi)

# Step 4: Matrix ko expand aur simplify karein
density_matrix = rho.doit().expand()
simplified_matrix = simplify(density_matrix)
simplified_matrix

α*Dagger(α)*|0>*<0| + α*Dagger(β)*|0>*<1| + β*Dagger(α)*|1>*<0| + β*Dagger(β)*|1>*<1|

In [6]:
res = represent(simplified_matrix.doit(), basis_set=[Qubit('0'), Qubit('1')])
res

# this is for one qubit, lets do for 2

Matrix([
[α*Dagger(α), α*Dagger(β)],
[β*Dagger(α), β*Dagger(β)]])

In [7]:
def get_rho_bipartite(input_qubit_tuple):
    q1a, q2a, q1b, q2b= map(lambda x: str(x), input_qubit_tuple)
    alpha, beta = symbols('α β')
    psi = alpha * TensorProduct(Qubit(q1a), Qubit(q2a)) + beta * TensorProduct(Qubit(q1b), Qubit(q2b))
    
    rho = psi * Dagger(psi)
    density_matrix = rho.doit().expand()
    simplified_matrix = simplify(density_matrix)
    print(simplified_matrix)
    
    rho = represent(simplified_matrix.doit(), basis_set=[Qubit('0'), Qubit('1')])
    return rho

In [8]:
dA, dB = 2, 2

In [9]:
def permute_dims(tensor, perm):
    """
    Permutes the dimensions of a sympy MutableDenseNDimArray according to the given permutation tuple.
    """
    old_shape = tensor.shape
    new_shape = tuple(old_shape[i] for i in perm)
    new_tensor = sp.MutableDenseNDimArray.zeros(*new_shape)
    for new_index in product(*[range(s) for s in new_shape]):
        # Compute the corresponding old index.
        old_index = [None] * len(new_index)
        for new_axis, old_axis in enumerate(perm):
            old_index[old_axis] = new_index[new_axis]
        new_tensor[new_index] = tensor[tuple(old_index)]
    return new_tensor


In [10]:
def compute_singular_vals_sympy(M):
    """
    Compute the singular values of matrix M by computing the eigenvalues of M^T*M.
    """
    MTM = M.T * M
    eig_dict = MTM.eigenvals()
    singular_vals = []
    for eigenvalue, multiplicity in eig_dict.items():
        # Simplify the eigenvalue for a cleaner expression.
        eigenvalue = sp.simplify(eigenvalue)
        # Assume eigenvalues are nonnegative; take the square root.
        s_val = sp.sqrt(eigenvalue)
        singular_vals.extend([sp.simplify(s_val)] * multiplicity)
    return singular_vals



In [11]:
def compute_entanglement_measures_symbolic(rho, dA, dB, display_output=True):
    """
    Computes entanglement measures for a bipartite system (dA x dB).
    """
    N = dA * dB
    assert rho.shape[0] == N, "Product dA*dB must equal the matrix dimension."
    
    # if display_output:
    #     display(Latex(r"Density matrix $\rho$: "))
    #     display(Latex(r"$$" + sp.latex(rho) + r"$$"))
    
    eig_dict_input = rho.eigenvals()
    eigenvals_input = [sp.simplify(ev) for ev, mult in eig_dict_input.items() for _ in range(mult)]
    # if display_output:
    #     display(Latex(r"Eigenvalues of $\rho$: "))
    #     display(Latex(r"$$" + sp.latex(eigenvals_input) + r"$$"))
    
    # Partial Transpose
    rho_tensor = sp.MutableDenseNDimArray(rho, (dA, dB, dA, dB))
    rho_pt_tensor = permute_dims(rho_tensor, (0, 3, 2, 1))
    rho_pt = sp.Matrix(rho_pt_tensor.reshape(N, N))
    # if display_output:
    #     display(Latex(r"Partial transpose $\rho^{T_B}$: "))
    #     display(Latex(r"$$" + sp.latex(rho_pt) + r"$$"))
    
    eig_dict_pt = rho_pt.eigenvals()
    eigenvals_pt = [sp.simplify(ev) for ev, mult in eig_dict_pt.items() for _ in range(mult)]
    # if display_output:
    #     display(Latex(r"Eigenvalues of $\rho^{T_B}$: "))
    #     display(Latex(r"$$" + sp.latex(eigenvals_pt) + r"$$"))
    
    det_rho_pt = sp.simplify(rho_pt.det())
    # if display_output:
    #     display(Latex(r"Determinant of $\rho^{T_B}$: "))
    #     display(Latex(r"$$" + sp.latex(det_rho_pt) + r"$$"))
    
    # Realignment
    rho_realigned_tensor = permute_dims(rho_tensor, (0, 2, 1, 3))
    realigned = sp.Matrix(rho_realigned_tensor.reshape(dA*dA, dB*dB))
    if display_output:
        display(Latex(r"Realigned matrix: "))
        display(Latex(r"$$" + sp.latex(realigned) + r"$$"))
    
    singular_vals = compute_singular_vals_sympy(realigned)
    # if display_output:
    #     display(Latex(r"Singular values of the realigned matrix: "))
    #     display(Latex(r"$$" + sp.latex(singular_vals) + r"$$"))
    
    sum_sv = sp.simplify(sum(singular_vals))
    if display_output:
        display(Latex(r"Sum of singular values: "))
        display(Latex(r"$$" + sp.latex(sum_sv) + r"$$"))
    
    # product_sv = sp.simplify(sp.Mul(*singular_vals))
    # if display_output:
    #     display(Latex(r"Product of singular values: "))
    #     display(Latex(r"$$" + sp.latex(product_sv) + r"$$"))
    
    # det_realigned = sp.simplify(realigned.det())
    # if display_output:
    #     display(Latex(r"Determinant of the realigned matrix: "))
    #     display(Latex(r"$$" + sp.latex(det_realigned) + r"$$"))
    
    # product_eigenvals = sp.simplify(sp.Mul(*eigenvals_pt))
    # if display_output:
    #     display(Latex(r"Product of eigenvalues of $\rho^{T_B}$: "))
    #     display(Latex(r"$$" + sp.latex(product_eigenvals) + r"$$"))
    
    #return eigenvals_input, eigenvals_pt, singular_vals, sum_sv, product_sv, product_eigenvals, det_rho_pt, det_realigned
    return realigned, sum_sv


In [12]:
realigned, sum_sv= compute_entanglement_measures_symbolic(get_rho_bipartite((0, 0, 1, 1)), dA, dB, False)

α*Dagger(α)*|0>x|0>*<0|x<0| + α*Dagger(β)*|0>x|0>*<1|x<1| + β*Dagger(α)*|1>x|1>*<0|x<0| + β*Dagger(β)*|1>x|1>*<1|x<1|


In [13]:
realigned

Matrix([
[α*Dagger(α),           0,           0,           0],
[          0, α*Dagger(β),           0,           0],
[          0,           0, β*Dagger(α),           0],
[          0,           0,           0, β*Dagger(β)]])

In [14]:
sum_sv

sqrt(α**2*Dagger(α)**2) + sqrt(α**2*Dagger(β)**2) + sqrt(β**2*Dagger(α)**2) + sqrt(β**2*Dagger(β)**2)

In [15]:
def generate_alpha_beta_imaginary():
    theta = random.uniform(0, math.pi / 2)
    phi = random.uniform(0, 2 * math.pi)
    psi = random.uniform(0, 2 * math.pi)
    alpha= math.cos(theta) * cmath.exp(1j * phi)
    beta= math.sin(theta) * cmath.exp(1j * psi)
    return alpha, beta

In [16]:
a, b= generate_alpha_beta_imaginary()

In [17]:
simplify(sum_sv.subs({alpha: a, beta: b}))

1.0 + 1.0*sqrt(-0.00594718300236594 - 0.00465912804344832*I) + 1.0*sqrt(-0.00594718300236594 + 0.00465912804344832*I)

In [18]:
def generate_alpha_beta_real():
    theta = random.uniform(0, math.pi / 2)
    alpha= math.cos(theta)
    beta= math.sin(theta)
    return alpha, beta

In [19]:
a, b= generate_alpha_beta_real()

In [20]:
simplify(sum_sv.subs({alpha: a, beta: b}))

1.20437727230726

In [21]:
string_length= 4
no_of_random_samples_for_each_bipartite_combinations= 20
combinations= list(itertools.product([0, 1], repeat= string_length))
for comb in combinations:
    # 0011, 0110, ... so on
    rho= get_rho_bipartite(comb)
    realigned, sum_sv= compute_entanglement_measures_symbolic(rho, dA, dB, False)
    for i in range(no_of_random_samples_for_each_bipartite_combinations):
        a, b= generate_alpha_beta_real()
        print(simplify(sum_sv.subs({alpha: a, beta: b})))
    

α*Dagger(α)*|0>x|0>*<0|x<0| + α*Dagger(β)*|0>x|0>*<0|x<0| + β*Dagger(α)*|0>x|0>*<0|x<0| + β*Dagger(β)*|0>x|0>*<0|x<0|
1.97290535592325
1.59917973346911
1.44833797173254
1.97252038262179
1.99729106590411
1.77694145427956
1.33855256245721
1.82644255698941
1.19028628105833
1.24776998204207
1.68753376758757
1.37614008234074
1.48623867070962
1.99579703361443
1.14833632399625
1.79662151749704
1.94838936248938
1.73869821266112
1.98757329275587
1.16159067138619
α*Dagger(α)*|0>x|0>*<0|x<0| + α*Dagger(β)*|0>x|0>*<0|x<1| + β*Dagger(α)*|0>x|1>*<0|x<0| + β*Dagger(β)*|0>x|1>*<0|x<1|
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
1.00000000000000
α*Dagger(α)*|0>x|0>*<0|x<0| + α*Dagger(β)*|0>x|0>*<1|x<0| + β*Dagger(α)*|1>x|0>*<0|x

In [22]:
# Bell state (|00⟩ + |11⟩)/√2
bell_state = (TensorProduct(Qubit('0'), Qubit('0')) + 
             TensorProduct(Qubit('1'), Qubit('1')))/sqrt(2)

# Density matrix
bell_rho = bell_state * Dagger(bell_state)
bell_rho

(|0>x|0> + |1>x|1>)*(<0|x<0| + <1|x<1|)/2

In [23]:
matrix_form = represent(bell_rho.doit(), basis_set=[Qubit('0'), Qubit('1')])

# Simplify करें और output को Matrix के रूप में प्राप्त करें
final_matrix = Matrix(simplify(matrix_form)).expand()
final_matrix

Matrix([
[1/2, 0, 0, 1/2],
[  0, 0, 0,   0],
[  0, 0, 0,   0],
[1/2, 0, 0, 1/2]])

In [24]:
# Bell state vector (|00⟩ + |11⟩)/√2 को Matrix के रूप में परिभाषित करें
bell_vector = Matrix([[1], [0], [0], [1]])/sqrt(2)

# Density matrix बनाएं (outer product)
density_matrix = bell_vector * Dagger(bell_vector)

# मैट्रिक्स को simplify करें और expand करें
rho = density_matrix.expand()
rho

Matrix([
[1/2, 0, 0, 1/2],
[  0, 0, 0,   0],
[  0, 0, 0,   0],
[1/2, 0, 0, 1/2]])

In [ ]:
# Execute the optimized function

result = compute_entanglement_measures_symbolic(rho, dA, dB)
result

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
ajeeb_matrix= sp.Matrix([
    [0.5, 0, 0, 0],
    [0, 0, 0.5, 0],
    [0, 0.5, 0, 0],
    [0, 0, 0, 0.5]
])
ajeeb_matrix


In [ ]:
ajeeb_matrix.det()

In [ ]:
1 / 16